# AutoMl Homework 2

## Filip Skrzeczkowski

## Filip Suchorab

In [5]:
# Read data

from data_loading import load_data

X_train, X_test, y_train, y_test, final_X_test = load_data(True)


In [3]:
# X_train.head()
# y_train.head()
final_X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,483,454,513,495,523,469,453,477,506,479,...,455,480,543,259,413,520,485,498,523,510
1,485,508,493,487,478,472,504,476,479,475,...,486,480,535,534,514,452,484,495,548,477
2,483,521,507,475,493,486,421,475,496,483,...,491,476,498,495,508,528,486,465,508,503
3,474,504,576,480,553,483,524,478,483,483,...,521,475,470,463,509,525,479,467,552,517
4,495,474,523,479,495,488,485,476,497,478,...,510,471,522,343,509,520,475,493,506,491


In [4]:
# Check if data is balanced (for balanced data balanced accuracy ~= accuracy)
class1 = 0
class2 = 0

for elem in y_train[0]:
    if (elem == 0):
        class1 += 1
    else:
        class2 += 1
    
print(class1)
print(class2)

1000
1000


## Model stworzony manualnie

In [ ]:
# find features to exclude
from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
feature_names = ['F{}'.format(i) for i in range(X_train.shape[1])]
train_pool = Pool(X_train, y_train, feature_names=feature_names)
test_pool = Pool(X_test, y_test, feature_names=feature_names)

model = CatBoostClassifier()
summary = model.select_features(
    train_pool,
    eval_set=test_pool,
    features_for_select='0-499',
    num_features_to_select=20,
    steps=5,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Regular,
    train_final_model=True,
    logging_level='Verbose',
    plot=True
)
print(summary['eliminated_features'])

In [24]:
# modify columns
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

class CustomDataModifier(BaseEstimator, ClassifierMixin):
    def __init__(self, classifier, data_modifier):
        self.classifier = classifier
        self.data_modifier = data_modifier

    def fit(self, X, y):
        # Modify the data before fitting
        X_modified = self.data_modifier(X)
        self.classifier.fit(X_modified, y)
        return self

    def predict_proba(self, X):
        # Modify the data before predicting probabilities
        X_modified = self.data_modifier(X)
        return self.classifier.predict_proba(X_modified)


    def predict(self, X):
        # Modify the data before making predictions
        X_modified = self.data_modifier(X)
        return self.classifier.predict(X_modified)

# Example data modifier function
def modify_data(X):
    # Implement your data modification logic here
    # For example, you can add a constant value to all features
    selected = [4, 281, 433, 318, 453, 455, 334, 217, 475, 105, 378]
    # [128, 384, 4, 281, 153, 28, 48, 433, 318, 451, 453, 455, 334, 217, 475, 226, 105, 378] 0.881
    # [12, 4, 281, 153, 28, 433, 318, 451, 453, 455, 334, 217, 475, 226, 105, 378] 0.882
    # [4, 281, 28, 433, 318, 453, 455, 334, 217, 475, 226, 105, 378] 0.8845
    # [4, 281, 28, 433, 318, 453, 455, 334, 217, 475, 105, 378] 0.8856
    # [4, 281, 433, 318, 453, 455, 334, 217, 475, 105, 378] 0.8895
    return X.iloc[:, selected]

In [ ]:
#lgbm
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(
    objective= 'binary', 
    num_leaves= 60, 
    num_iterations= 150, 
    min_data_in_leaf= 10, 
    metric= 'binary_logloss', 
    max_bin= 500, 
    learning_rate= 0.03,
    random_state=420,
)
lgbm = CustomDataModifier(lgbm, data_modifier=modify_data)
scores = scores = cross_val_score(lgbm, X_train, y_train, cv=5, scoring=make_scorer(balanced_accuracy_score)) 

In [26]:
print("Cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())

Cross-validation scores: [0.8425 0.89   0.8575 0.8775 0.8625]
Mean accuracy: 0.866


In [29]:
#xgboost
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
xgb = XGBClassifier(eval_metric="error", 
                    n_estimators=85, 
                    colsample_bytree=0.602,
                    gamma=6,
                    learning_rate=0.0312,
                    max_depth=10,
                    min_child_weight=1,
                    subsample=0.97,
                    random_state=420,
)
xgb = CustomDataModifier(xgb, data_modifier=modify_data)
scores = scores = cross_val_score(xgb, X_train, y_train, cv=5, scoring=make_scorer(balanced_accuracy_score)) 

In [30]:
print("Cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())

Cross-validation scores: [0.7875 0.8375 0.8025 0.8275 0.83  ]
Mean accuracy: 0.8170000000000002


In [ ]:
#catboost
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier

cat = CustomDataModifier(CatBoostClassifier(), data_modifier=modify_data)
scores = scores = cross_val_score(cat, X_train, y_train, cv=5, scoring=make_scorer(balanced_accuracy_score)) 

In [38]:
print("Cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())

Cross-validation scores: [0.885  0.9025 0.8725 0.9025 0.885 ]
Mean accuracy: 0.8895


Final Model

In [40]:
from catboost import CatBoostClassifier
from data_loading import save_data

model = CustomDataModifier(CatBoostClassifier(), data_modifier=modify_data)
model.fit(X_train, y_train)
proba = model.predict_proba(final_X_test)
save_data(proba, "313453_313457_artifical_model_prediction.txt")

Learning rate set to 0.013851
0:	learn: 0.6837183	total: 3.22ms	remaining: 3.22s
1:	learn: 0.6745557	total: 5.81ms	remaining: 2.9s
2:	learn: 0.6666430	total: 8.39ms	remaining: 2.79s
3:	learn: 0.6616899	total: 11.7ms	remaining: 2.91s
4:	learn: 0.6536708	total: 14.2ms	remaining: 2.83s
5:	learn: 0.6444326	total: 18.2ms	remaining: 3.01s
6:	learn: 0.6364179	total: 20.5ms	remaining: 2.91s
7:	learn: 0.6278247	total: 23.3ms	remaining: 2.88s
8:	learn: 0.6209104	total: 25.8ms	remaining: 2.84s
9:	learn: 0.6145480	total: 28.8ms	remaining: 2.85s
10:	learn: 0.6078152	total: 31.1ms	remaining: 2.8s
11:	learn: 0.6005797	total: 33.3ms	remaining: 2.74s
12:	learn: 0.5934400	total: 35.3ms	remaining: 2.68s
13:	learn: 0.5869750	total: 37.4ms	remaining: 2.63s
14:	learn: 0.5802549	total: 39.4ms	remaining: 2.58s
15:	learn: 0.5736899	total: 41.4ms	remaining: 2.54s
16:	learn: 0.5690546	total: 44.3ms	remaining: 2.56s
17:	learn: 0.5628961	total: 46.7ms	remaining: 2.55s
18:	learn: 0.5566445	total: 48.9ms	remaining: 

## AutoML

In [4]:
from supervised import AutoML

ml = AutoML(eval_metric='accuracy', mode="Compete", stack_models=True, total_time_limit=60*60)
ml.fit(X_train, y_train)


/home/filip/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AutoML directory: AutoML_1
The task is binary_classification with evaluation metric accuracy
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree accuracy 0.68 trained in 2.84 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 4 models
1_DecisionTree accuracy 0.7055 trained in 6.48 seconds
2_DecisionTree accuracy 0.763 trained in 6.75 seconds
3_DecisionTree accuracy 0.762 trained in 6.72 seconds


/home/filip/.local/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/home/filip/.local/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/home/filip/.local/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/home/filip/.local/lib/python3.10/site-packages/xgbo

6_Default_Xgboost accuracy 0.8235 trained in 26.54 seconds
7_Default_CatBoost accuracy 0.865 trained in 25.91 seconds
8_Default_NeuralNetwork accuracy 0.5555 trained in 29.35 seconds


/home/filip/.local/lib/python3.10/site-packages/supervised/utils/learning_curves.py:113: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
/home/filip/.local/lib/python3.10/site-packages/supervised/utils/learning_curves.py:113: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
/home/filip/.local/lib/python3.10/site-packages/supervised/utils/learning_curves.py:113: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
/home/filip/.local/lib/python3.10/site-packages/supervised/utils/learning_curves.py:113: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
/home/filip/

...

/home/filip/.local/lib/python3.10/site-packages/supervised/utils/learning_curves.py:113: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
/home/filip/.local/lib/python3.10/site-packages/supervised/utils/learning_curves.py:113: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
/home/filip/.local/lib/python3.10/site-packages/supervised/utils/learning_curves.py:113: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
/home/filip/.local/lib/python3.10/site-packages/supervised/utils/learning_curves.py:113: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
/home/filip/

69_CatBoost_Stacked accuracy 0.8805 trained in 71.27 seconds
27_LightGBM_Stacked accuracy 0.882 trained in 17.49 seconds
48_ExtraTrees_Stacked accuracy 0.8755 trained in 37.02 seconds
43_RandomForest_Stacked not trained. Stop training after the first fold. Time needed to train on the first fold 5.0 seconds. The time estimate for training on all folds is larger than total_time_limit.


/home/filip/.local/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/home/filip/.local/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/home/filip/.local/lib/python3.10/site-packages/xgboost/training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/home/filip/.local/lib/python3.10/site-packages/xgbo

12_Xgboost_Stacked accuracy 0.8785 trained in 33.43 seconds
61_NeuralNetwork_Stacked not trained. Stop training after the first fold. Time needed to train on the first fold 1.0 seconds. The time estimate for training on all folds is larger than total_time_limit.
35_CatBoost_Stacked not trained. Stop training after the first fold. Time needed to train on the first fold 6.0 seconds. The time estimate for training on all folds is larger than total_time_limit.
* Step ensemble_stacked will try to check up to 1 model
Ensemble_Stacked accuracy 0.884 trained in 6.89 seconds
AutoML fit time: 3620.29 seconds
AutoML best model: 67_CatBoost_GoldenFeatures_Stacked


AutoML(eval_metric='accuracy', mode='Compete', stack_models=True)

In [8]:

proba = ml.predict_proba(final_X_test)
print(proba)


[[0.67180094 0.32819906]
 [0.68392419 0.31607581]
 [0.55121206 0.44878794]
 ...
 [0.3123336  0.6876664 ]
 [0.31183161 0.68816839]
 [0.70084614 0.29915386]]


AttributeError: module 'data_loading' has no attribute 'save_data'

In [14]:
from data_loading import save_data
save_data(proba, "313453_313457_artifical_automl_prediction.txt")